In [75]:
from abc import ABC, abstractmethod
import re
from logging import DEBUG, StreamHandler, getLogger

from openai import OpenAI

In [2]:
# Set logging level to DEBUG.
logger = getLogger(__name__)
logger.setLevel(DEBUG)
handler = StreamHandler()
handler.setLevel(DEBUG)
logger.addHandler(handler)

In [ ]:
class LanguageModel(ABC):
    def __init__(self, model_name: str, temperature=0.7, max_tokens=512, seed=0):
        self.model_name = model_name
        self.temperature = temperature
        self.max_tokens = max_tokens
        self.seed = seed
        logger.debug(f"model_name: {model_name}, temperature: {temperature}, max_tokens: {max_tokens}, seed: {seed}")

    @abstractmethod
    def __call__(self, messages_list: list[list[dict]]) -> dict[str, list[str]]:
        pass

In [86]:
# You must set OPENAI_API_KEY in your environment variables.
class OpenAIAPI(LanguageModel):
    def __init__(self, model_name: str, temperature=0.7, max_tokens=512, seed=0, base_url=None, api_key=None):
        super().__init__(model_name, temperature=temperature, max_tokens=max_tokens, seed=seed)
        self.client = OpenAI(base_url=base_url, api_key=api_key)
        logger.debug(f"base_url: {self.client.base_url}, api_key: {self.client.api_key[:3]}...")

    def __call__(self, messages_list: list[list[dict]]) -> dict[str, list[str]]:
        contents: list[str] = []
        for messages in messages_list:
            completion = self.client.chat.completions.create(
                model=self.model_name,
                messages=messages,
                temperature=self.temperature,
                max_tokens=self.max_tokens,
                seed=self.seed,
            )
            contents.append(completion.choices[0].message.content or "")
        return {"contents": contents}

In [87]:
lm = OpenAIAPI("gpt-4o-mini", temperature=0.7, max_tokens=512, seed=0)
lm

model_name: gpt-4o-mini, temperature: 0.7, max_tokens: 512, seed: 0
base_url: https://api.openai.com/v1/, api_key: sk-...


In [89]:
def get_q1_prompt(task: str, topic: str, item: str) -> list[dict]:
    Q1_PROMPT_TEMPLATE = """Create a {task} problem related to the following {topic}:

{item}

Note:

1. The {task} problem should be simple and involve basic {task} skills and knowledge. Any average grade school student can solve it correctly.
2. You should make full use of the {topic} description to create the {task} problem to ensure that the {task} problem is unique and specific to the {topic}.
3. Your response should always start with "問題:". Your response should not include a solution to the created {task} problem.
4. 簡潔に日本語で回答してください。
"""
    return [
        # {"role": "system", "content": "あなたは親切なAIアシスタントです。日本語で回答してください。"},
        {"role": "user", "content": Q1_PROMPT_TEMPLATE.format(task=task, topic=topic, item=item)},
    ]

In [62]:
get_q1_prompt("math", "persona", "SF作家")

[{'role': 'user',
  'content': 'Create a math problem related to the following persona:\n\nSF作家\n\nNote:\n\n1. The math problem should be simple and involve basic math skills and knowledge. Any average grade school student can solve it correctly.\n2. You should make full use of the persona description to create the math problem to ensure that the math problem is unique and specific to the persona.\n3. Your response should always start with "問題:". Your response should not include a solution to the created math problem.\n4. 簡潔に日本語で回答してください。\n'}]

In [65]:
lm([get_q1_prompt("math", "persona", "SF作家")])

{'contents': ['問題: SF作家の佐藤さんは、新しい短編小説を3つ書くことに決めました。1つの短編小説を書くのに、彼は平均して2時間かかります。佐藤さんが全ての短編小説を書くのにかかる合計時間は何時間ですか？']}

In [66]:
lm([get_q1_prompt("math", "persona", "経営コンサルタント")])

{'contents': ['問題: 経営コンサルタントの佐藤さんは、クライアントの会社の売上を増やすために3つの異なるマーケティング戦略を提案しました。最初の戦略では、売上が月に200万円増えると予想されています。2つ目の戦略では、売上が月に150万円増えると予想されています。3つ目の戦略では、売上が月に250万円増えると予想されています。もし、佐藤さんが全ての戦略を同時に実施した場合、1ヶ月後の売上の増加額は合計でいくらになるでしょうか？']}

In [90]:
def generate_q1(lm, tasks, topics, items):
    response = lm([get_q1_prompt(task, topic, item) for task, topic, item in zip(tasks, topics, items)])
    contents = []
    for content in response["contents"]:
        content = content.strip()
        content = re.sub(r"^問題[:]", "", content, flags=re.DOTALL)
        content = content.strip()
        contents.append(content)
    return {"contents": contents}


In [92]:
q1_response = generate_q1(lm, ["math", "math"], ["persona", "persona"], ["SF作家", "経営コンサルタント"])
q1_response

{'contents': ['SF作家の田中さんは、新しい短編小説を3つ書くことに決めました。1つの短編小説を書くのに、田中さんは平均して4時間かかります。もし田中さんが毎日2時間ずつ執筆する場合、3つの短編小説を書くのに何日かかりますか？',
  '経営コンサルタントの佐藤さんは、クライアントの会社の売上を増やすために3つの異なるマーケティング戦略を提案しました。最初の戦略では、売上が月に200万円増えると予想されています。2つ目の戦略では、売上が月に150万円増えると予想されています。3つ目の戦略では、売上が月に250万円増えると予想されています。もし、佐藤さんが全ての戦略を同時に実施した場合、1ヶ月後の売上の増加額は合計でいくらになるでしょうか？']}

In [ ]:
def get_a1_prompt(q1: str) -> list[dict]:
    A1_PROMPT_TEMPLATE = "{q1}\n\n簡潔に日本語で回答してください。"
    return [
        # {"role": "system", "content": "あなたは親切なAIアシスタントです。日本語で回答してください。"},
        {"role": "user", "content": A1_PROMPT_TEMPLATE.format(q1=q1)},
    ]


In [93]:
get_a1_prompt(q1_response["contents"][0])

[{'role': 'user',
  'content': 'SF作家の田中さんは、新しい短編小説を3つ書くことに決めました。1つの短編小説を書くのに、田中さんは平均して4時間かかります。もし田中さんが毎日2時間ずつ執筆する場合、3つの短編小説を書くのに何日かかりますか？\n\n簡潔に日本語で回答してください。'}]

In [95]:
lm([get_a1_prompt(q1_response["contents"][0])])

{'contents': ['田中さんが3つの短編小説を書くのにかかる時間は、4時間 × 3つ = 12時間です。毎日2時間ずつ執筆するので、12時間 ÷ 2時間/日 = 6日かかります。したがって、3つの短編小説を書くのに6日かかります。']}